In [1]:
# Setup
%run Util.ipynb
%run AuditingMethods.ipynb

,0.01,0.02,0.05,0.1,0.2,0.5
100,2.683,2.500,2.236,2.000,1.732,1.155
300,2.887,2.694,2.425,2.145,1.877,1.343
1000,3.054,2.864,2.546,2.294,2.000,1.414
3000,3.184,3.000,2.670,2.401,2.095,1.511
10000,3.290,3.077,2.770,2.496,2.183,1.633
30000,3.357,3.144,2.828,2.556,2.240,1.715
100000,3.411,3.206,2.889,2.638,2.324,1.747
300000,3.487,3.273,2.958,2.684,2.375,1.817
1000000,3.530,3.309,3.000,2.734,2.438,1.890
3000000,3.560,3.352,3.040,2.782,2.474,1.937


# Process as a stochastic process

In [2]:
from collections import defaultdict as dd
from collections import deque
import sys

''' Turn a function to class instance of Cached class and 
    cache result whenever any one calles the new function

    In this Example it is used for ackermann function
    Note here, the wrapped function is no longer a Function, but rather a
    instance of class Cached. 

    1. ack = Cached(ack(x, y)) --> already a instance of Cached
       1.1 ack.function = ack(x, y)
       1.2 ack.cache = {}
    2. ack(1, 1) -> calls the __call__ method because its an instance
    3. the dictionary is filled with the recursive calls

    (4. imagine using this for a prime number finder, it can auto cache the
        prime number found, and the time of calculation is greatly shortened
        by refereing to previous function results in dictionary)
'''


class Cached:
    def __init__(self, function):
        self.function = function
        self.cache = {}

    def __call__(self, *args, **kwargs):
        sorted_kwargs = tuple(sorted(kwargs.items()))
        try:
            cached_value = self.cache[(args, sorted_kwargs)]
            # print("cached")
            return cached_value
        
        except KeyError:
            ret = self.cache[(args, sorted_kwargs)] = self.function(*args, **kwargs)
            return ret
        

class OrderedDict:
    
    def __init__(self):
        # Priority queue for keys
        self._q = deque()
        
        # Dictionary for values
        self._dict = dd(float)
        
    def append(self, key, value):
        # Didn't find
        if key not in self._q:
            self._q.appendleft(key)
            self._dict[key] = value
        else:
            self._dict[key] += value
        
    def pop(self):
        key = self._q.pop()
        value = self._dict[key]
        del self._dict[key]
        return key, value

    def peek(self):
        key = self._q[-1]
        value = self._dict[key]
        return key, value
    
    def __repr__(self):
        return str([(key, self._dict[key]) for key in self._q])
    
    def serialize(self):
        return [(key, self._dict[key]) for key in self._q]
    

class SimpleProgressionBar:
    _prefix_format = "{0:6}/{1:6}"
    _bar_format = "|{:<20}|"
    
    def __init__(self, n):
        self._n = n
        self._prev = ""
        
    def __call__(self, x):
        solid = x*20//self._n
        prefix = self._prefix_format.format(x, self._n)
        bar = self._bar_format.format(u"\u25A0" * solid + u"\u25A1" * (20-solid))
        self._prev = prefix + bar
        print("\r" + self._prev, end="")
        if x == self._n:
            print()

def simple_bfs2(n, m=100, step=1, a=1, b=1, thresh=0.95, p_h0=1/2):
    if callable(thresh):
        thresh_fn = thresh
        thresh = thresh_fn(0)
        print(thresh, thresh_fn)
    
    progression_bar = SimpleProgressionBar(m)
    
    # Should be able to use depth first search and dynamic programming
    depth = 0
    
    risk_dict = dd(float)
    
    cached_binom_pmf = Cached(binom.pmf)
    cached_beta_binomial_cdf = Cached(beta_binomial_cdf)
    
    # prob_dict = dd(float)
    
    # first element: sa_t,
    # second element: s_t
    # third element: the chance going to this state
    source = (0, 0, 1)
    
    q = OrderedDict()
    q.append(source[:2], source[2])
    old_depth = depth
    rejection_states = set()
    while depth <= m:
        if old_depth != depth:
            old_depth = depth
            progression_bar(depth)
            
        # get next node to explore
        key, value = q.pop()
        sa, s, p_t = *key, value
        depth = s // step
        try:
            thresh = thresh_fn(s)
        except NameError:
            pass

        # All possible generated sa for next batch
        for i in range(step+1):
            # the binomial probability reaching next node
            p_binom = cached_binom_pmf(i, step, p_h0)

            # rejection probability
            p_reject = 1 - cached_beta_binomial_cdf(n/2-sa-i, sa+a+i, b+(s+step-sa-i), n-s-step)

            # Compose the node
            node = (sa+i, s+step, p_binom * p_t)

            # if null is rejected, put it in the risk dict
            if p_reject > thresh:
                risk_dict[node[:2]] += node[2]
            else:
                q.append(node[:2], node[2])
    return risk_dict, q

In [3]:
import scipy as sp
import numpy as np
import pandas as pd
from functools import partial


@Cached
def index_conversion(t, y_t, step):
    """
    Convert the (t, y_t) coordinate to matrix coordinate
    """
    #    1        step*1+1          step*2+1         step*3+1          1   ....         t + 1
    # [0, 0], [step*1+1, y_1], [step*2+1, y_2], [step*3+1, y_3] ... (t, y_t(0)) ... (t, y_t(~))
    # So there are 1 + 1*step+1 + step*2+1 +... + step*(t-1)+1 + y_t number before
    # the specified entry, the entry is at
    return step * ((t-1)*t)//2 + t + y_t # (+1-1) due to 0 index


@Cached
def reverse_conversion(step, index):
    t = 0
    total = 0
    while total < index:
        t += 1
        total += (t * step + 1)
    return t, t* step - (total - index)


@Cached
def binom_pmf(k, n, p):
    return binom.pmf(k, n, p)
    

def find_terminal(f, n, m=1, step=10, p_h0=1/2, to_sequence=False, transition=False):
    """
    Args:
        f (callable): return 1 if the null is rejected else 0"""
    # 1 + step+1 + 2*step+1 + 3*step+1 +... + m*step+1 = step*((m-1)*m)/2 + (m+1)
    s_total = step * ((m+1)*m)//2 + (m+1)
    rejections = set()
    
    # Initialise a DataFrame as dense matrix
    indexes = []
    for t in range(m + 1):
        total = t * step
        for y_t in range(total + 1):
            indexes.append(str((t * step, y_t)))
    transition_matrix = pd.DataFrame(0, columns=indexes, index=indexes)
    
    for t in range(m + 1):
        total = t * step
        for y_t in range(total + 1):
            index = str((total, y_t))
            # Rejected?
            if f(n, total, y_t):
                transition_matrix.loc[index, index] = 1
                rejections.add(index)
            # Already last layer
            elif t == m:
                transition_matrix.loc[index, index] = 1
            else:                    
                t_next = t + 1
                for y_t_add in range(step+1):
                    y_t_next = y_t + y_t_add
                    index_next = str((t_next * step, y_t_next))
                    transition_matrix.loc[index, index_next] = binom_pmf(y_t_add, step, p_h0)
    if transition:
        return transition_matrix
    # Initialise the sparse matrix to sovle this chain
    # transition_matrix = sp.sparse.csr_matrix(transition_matrix)
    stationary = solve_stationary(transition_matrix.values)
    d = {indexes[i]: np.asscalar(stationary[i]) for i in range(len(indexes))}
    if to_sequence:
        return pd.Series(d), rejections
    return d, rejections


def convert_to_sequence(array, m, step):
    d = {}
    for t in range(m + 1):
        for y_t in range(t*step + 1):
            index = index_conversion(t, y_t, step)
            d[(t * step, y_t)] = np.asscalar(array[index])
    return pd.Series(d)
    
    
def solve_stationary(A):
    """ x = xA where x is the answer
    x - xA = 0
    x( I - A ) = 0 and sum(x) = 1
    """
    n = A.shape[0]
    a = np.eye(n) - A
    a = np.vstack((a.T, np.ones(n)))
    b = np.matrix([0] * n + [1]).T
    return sp.linalg.lstsq(a, b)[0]

In [4]:
beta_binomial_cdf = Cached(beta_binomial_cdf)
@Cached
def bayesian_f(n, t, y_t, a=1, b=1, thresh=0.95):
    k = ceil(n/2 - y_t)
    p_reject = 1 - beta_binomial_cdf(k, y_t+a, t-y_t+b, n-t)
    return p_reject > thresh

In [5]:
def stochastic_process_simulation(rejection_fn, n, m=1000, step=1, p=1/2, progression=False, *args, **kwargs):  
    progression_bar = lambda x:None
    
    if progression:
        progression_bar = SimpleProgressionBar(m)
    
    depth = 0
    
    risk_dict = dd(float)
    
    cached_binom_pmf = Cached(binom.pmf)
    
    # first element: t,
    # second element: y_t, (observe every step time)
    # third element: probability going to this state
    source = (0, 0, 1)
    
    q = OrderedDict()
    q.append(source[:2], source[2])
    old_depth = depth

    rejection_states = set()
    while True:                
        # get next node to explore
        key, value = q.peek()
        
        progression_bar(key[0])
        
        if key[0] >= m:
            break
        q.pop()
        t, y_t, p_t = *key, value

        t_next = t + step
        # All possible generated sa for next batch
        for i in range(step+1):
            # the binomial probability reaching next node
            p_binom = cached_binom_pmf(i, step, p)
            
            y_t_next = y_t + i
            
            # rejection?
            reject = rejection_fn(n, t_next, y_t_next, *args, **kwargs)

            # Compose the node
            node = (t_next, y_t_next, p_binom * p_t)

            # if null is rejected, put it in the risk dict
            if reject:
                rejection_states.add(node[:2])
                risk_dict[node[:2]] += node[2]
            else:
                q.append(node[:2], node[2])
    return risk_dict, rejection_states, q

In [6]:
stochastic_process_simulation(lambda x,y,z:False, n=1000, m=200, step=1, p=1/2, progression=False)

(defaultdict(float, {}),
 set(),
 [((200, 200), 6.223015277861142e-61), ((200, 199), 1.2446030555722283e-58), ((200, 198), 1.2383800402943672e-56), ((200, 197), 8.173308265942824e-55), ((200, 196), 4.0253543209768406e-53), ((200, 195), 1.5779388938229215e-51), ((200, 194), 5.128301404924495e-50), ((200, 193), 1.421272103650503e-48), ((200, 192), 3.428818950056838e-47), ((200, 191), 7.314813760121255e-46), ((200, 190), 1.3971294281831597e-44), ((200, 189), 2.4132235577709125e-43), ((200, 188), 3.8008271034891865e-42), ((200, 187), 5.496580734276669e-41), ((200, 186), 7.3418614093552625e-40), ((200, 185), 9.103908147600526e-39), ((200, 184), 1.0526393795663111e-37), ((200, 183), 1.1393273284717721e-36), ((200, 182), 1.1583161172796351e-35), ((200, 181), 1.1095449123415453e-34), ((200, 180), 1.0041381456690984e-33), ((200, 179), 8.606898391449416e-33), ((200, 178), 7.00288550940657e-32), ((200, 177), 5.419624437714649e-31), ((200, 176), 3.996973022814554e-30), ((200, 175), 2.8138690080614

In [7]:
which python3

/anaconda/envs/ml_env/bin/R
